In [142]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [143]:
 # for project testing - read in csv, drop some attributes, shuffle data and form training & testing set    
testfile = "realdata.csv"
#read In Training data
data = pd.read_csv("clean_data.csv");
data.gender.replace('male',0, inplace=True)
data.gender.replace('female',1, inplace=True)
trainLabels = data['gender'].values            #stores all the labels
#utilize all data
drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text']
#utilize everything but text data
#drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text',
#                   'LMalecount', 'LFemalecount', 'Lintersectcount','SMalecount', 'SFemalecount', 'Sintersectcount']
traindata = data.drop(drop_attributes,axis=1)   # store all the data

#Read In Test Data
test_data = pd.read_csv(testfile)
 #set up data for classifying
test_data.gender.replace('male',0, inplace=True)
test_data.gender.replace('female',1, inplace=True)
shuffle = test_data.sample(frac=1).reset_index(drop=True) #shuffle data twice
test_data = shuffle.sample(frac=1).reset_index(drop=True)
testLabels = test_data['gender'].values                   #stores test labels
drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text']
#drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text',
#                   'LMalecount', 'LFemalecount', 'Lintersectcount','SMalecount', 'SFemalecount', 'Sintersectcount']
testdata = test_data.drop(drop_attributes,axis=1)        #stores test data


In [144]:
#knn & bagging project validation -uses text data -
def knnTest():
    knn = KNeighborsClassifier(n_neighbors=9)
    knn.fit(traindata.values,trainLabels)
    knn_score = np.mean(cross_val_score(knn,testdata.values,testLabels ,cv=10))
    print("knn score : " + str(knn_score) + " with neighbors = " + str(9))
    
    knn = KNeighborsClassifier(n_neighbors=9)
    knn_bag = BaggingClassifier(knn,max_samples=0.5, max_features=0.5)
    knn_bag.fit(traindata.values,trainLabels)
    knn_bag_score = np.mean(cross_val_score(knn_bag,testdata.values,testLabels ,cv=10))
    print("knn bag score : " + str(knn_bag_score) + " with neighbors = " + str(9))

In [145]:
#Random forest, adaboost with random forest, adaboost with decisiontree
def randForest():
        randf = RandomForestClassifier(min_samples_leaf = 300)
        randf.fit(traindata.values,trainLabels)
        randf_score = np.mean(cross_val_score(randf,testdata.values,testLabels ,cv=10))
        print("RandForest Gini score : " + str(randf_score))
        
        randE = RandomForestClassifier(criterion  = 'entropy',min_samples_leaf = 300)
        randE.fit(traindata.values,trainLabels)
        randE_score = np.mean(cross_val_score(randE,testdata.values,testLabels ,cv=10))
        print("RandForest Entropy score : " + str(randE_score))
        
        randf = RandomForestClassifier(min_samples_leaf = 300)
        ada = AdaBoostClassifier(base_estimator = randf)
        ada.fit(traindata.values,trainLabels)
        ada_score = np.mean(cross_val_score(ada,testdata.values,testLabels ,cv=10))
        print("Adaboost with RandForest Gini score : " + str(ada_score))
        
        randE = RandomForestClassifier(criterion  = 'entropy',min_samples_leaf = 300)
        adaE = AdaBoostClassifier(base_estimator = randE)
        adaE.fit(traindata.values,trainLabels)
        adaE_score = np.mean(cross_val_score(adaE,testdata.values,testLabels ,cv=10))
        print("Adaboost with RandForest Entropy score : " + str(adaE_score))
        
        ada_decision = AdaBoostClassifier()
        ada_decision.fit(traindata.values,trainLabels)
        ada_dec_score = np.mean(cross_val_score(ada_decision,testdata.values,testLabels ,cv=10))
        print("Adaboost with Decision Tree score : " + str(ada_dec_score))

In [146]:
#GaussianNB,MultinomialNB,MultinomialNB with adaboost
def bayes():
    gnb = GaussianNB()
    gnb.fit(traindata.values,trainLabels)
    gnb_score = np.mean(cross_val_score(gnb,testdata.values,testLabels ,cv=10))
    print("GaussianNaiveBayes score: " + str(gnb_score))
    
    mnb = MultinomialNB()
    mnb.fit(traindata.values,trainLabels)
    mnb_score = np.mean(cross_val_score(mnb,testdata.values,testLabels ,cv=10))
    print("MultinomialNaiveBayes score: " + str(mnb_score))
    
    mnb = MultinomialNB()
    ada = AdaBoostClassifier(base_estimator = mnb)
    ada.fit(traindata.values,trainLabels)
    ada_score = np.mean(cross_val_score(ada,testdata.values,testLabels ,cv=10)) 
    print("Adaboost with MultinomialNaiveBayes score : " + str(ada_score))    

In [147]:
#SVC
def svmTest():
    svc = SVC(gamma='auto')
    svc.fit(traindata.values,trainLabels)
    svc_score = np.mean(cross_val_score(svc,testdata.values,testLabels ,cv=10))
    print("svc score: " + str(svc_score))
    


In [148]:
# for analysis purposes
def knnAnalysis():
    for testSize in range(1,10):
        for n in range(1,10):
            train_set, test_set, train_labels, test_label= train_test_split( analysis_data.values, analysis_label, test_size=testSize/10)   
            knn = KNeighborsClassifier(n_neighbors=n)
            knn.fit(train_set,train_labels)
            curr_score = np.mean(cross_val_score(knn,test_set,test_label ,cv=10))
            #uncomment to manually find out of 100-the best size
            print("knn score : " + str(curr_score)  + " for testsize of " + str(testSize/10) + " with neighbors = " + str(n))
             
    #bag using optimal size from above, 9 neighbors is the best        
    train_set, test_set, train_labels, test_label= train_test_split( analysis_data.values, analysis_label, test_size=0.6)   
    for i in range(1,10):
        knn = KNeighborsClassifier(n_neighbors=i)
        knn_bag = BaggingClassifier(knn,max_samples=0.5, max_features=0.5)
        knn_bag.fit(train_set,train_labels)
        prediction = knn_bag.predict(test_set)
        print("knn bag score : " + str(accuracy_score(prediction,test_label)) + " for neighbors = " + str(i))
            


In [149]:
def randForestAnalysis():
    train_set, test_set, train_labels, test_label= train_test_split( analysis_data.values, analysis_label, test_size=0.6)   
    randGini = RandomForestClassifier()
    randEntropy = RandomForestClassifier(criterion  = 'entropy')
    randGini.fit(train_set,train_labels)
    randEntropy.fit(train_set,train_labels)
    randGini_score = np.mean(cross_val_score(randGini,test_set,test_label ,cv=10))
    randEntropy_score = np.mean(cross_val_score(randEntropy,test_set,test_label ,cv=10))
    print("RandomForest Gini score : " + str(randGini_score))
    print("RandomForest Entropy score: " + str(randEntropy_score))
    
    for n in range(1,11):
        randGini = RandomForestClassifier(min_samples_leaf = n*100)
        randEntropy = RandomForestClassifier(criterion  = 'entropy',min_samples_leaf = n*100) 
        randGini.fit(train_set,train_labels)
        randEntropy.fit(train_set,train_labels)
        randGini_score = np.mean(cross_val_score(randGini,test_set,test_label ,cv=10))
        randEntropy_score = np.mean(cross_val_score(randEntropy,test_set,test_label ,cv=10))
        print("RandomForest Gini score with min leaf " + str(n*100) + " = " + str(randGini_score))
        print("RandomForest Entropy score with min leaf "+ str(n*100) + " = " + str(randEntropy_score))

    randf = RandomForestClassifier(min_samples_leaf = 300)
    ada = AdaBoostClassifier(base_estimator = randf)
    ada.fit(train_set,train_labels)
    ada_score = np.mean(cross_val_score(ada,test_set,test_label ,cv=10))
    print("Adaboost with RandForest Gini score : " + str(ada_score))
        
    randE = RandomForestClassifier(criterion  = 'entropy',min_samples_leaf = 300)
    adaE = AdaBoostClassifier(base_estimator = randE)
    adaE.fit(train_set,train_labels)
    adaE_score = np.mean(cross_val_score(adaE,test_set,test_label,cv=10))
    print("Adaboost with RandForest Entropy score : " + str(adaE_score))

    ada_decision = AdaBoostClassifier()
    ada_decision.fit(train_set,train_labels)
    ada_dec_score = np.mean(cross_val_score(ada_decision,test_set,test_label,cv=10))
    print("Adaboost with Decision Tree score : " + str(ada_dec_score))

In [150]:
def bayesAnalysis():
    train_set, test_set, train_labels, test_label= train_test_split( analysis_data.values, analysis_label, test_size=0.6) 
    
    gnb = GaussianNB()
    gnb.fit(train_set,train_labels)
    gnb_score = np.mean(cross_val_score(gnb,test_set,test_label ,cv=10))
    print("GaussianNB score : " + str(gnb_score))
    
    mnb = MultinomialNB()
    mnb.fit(train_set,train_labels)
    mnb_score = np.mean(cross_val_score(mnb,test_set,test_label ,cv=10))
    print("MultinomialNB score : " + str(mnb_score))

In [151]:
def svcAnalysis():
    train_set, test_set, train_labels, test_label= train_test_split( analysis_data.values, analysis_label, test_size=0.6) 
    svc = SVC(gamma='auto')
    svc.fit(train_set,train_labels)
    svc_score = np.mean(cross_val_score(svc,test_set,test_label ,cv=10))
    print("SVC : " + str(svc_score))

In [152]:
#For project grade -just take the highest accuracy among the classifers
knnTest()
randForest()
bayes()
svmTest()

#everything below is for analysis purposes

knn score : 0.903752803988 with neighbors = 9
knn bag score : 0.912129407449 with neighbors = 9
RandForest Gini score : 0.923994443141
RandForest Entropy score : 0.923140888015
Adaboost with RandForest Gini score : 0.920272310126
Adaboost with RandForest Entropy score : 0.921668681329
Adaboost with Decision Tree score : 0.924227542533
GaussianNaiveBayes score: 0.92570119256
MultinomialNaiveBayes score: 0.923063669331
Adaboost with MultinomialNaiveBayes score : 0.618040485684
svc score: 0.924148880509


In [153]:
#Code for running analysis
#ctrl / for uncomment
# analysis_data = pd.read_csv("clean_data.csv");
# analysis_data.gender.replace('male',0, inplace=True)
# analysis_data.gender.replace('female',1, inplace=True)
# analysis_label = analysis_data['gender'].values
# #utilize all data
# drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text']

# #use lemmatization of text only
# #drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text',
# #                   'LMalecount', 'LFemalecount', 'Lintersectcount']

# #use stemming of text only
# # drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text',
# #                    'SMalecount', 'SFemalecount', 'Sintersectcount']

# #don't use text data
# #drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text',
# #                    'LMalecount', 'LFemalecount', 'Lintersectcount','SMalecount', 'SFemalecount', 'Sintersectcount']

# #use text data only
# # drop_attributes = ['gender','name','Lemma_description', 'Lemma_text','stem_description', 'stem_text',
# #                    'fav_number','tweet_count', 'sred', 'sgreen','sblue', 'lred', 'lgreen', 'lblue']


# analysis_data = analysis_data.drop(drop_attributes,axis=1)
# knnAnalysis()
# randForestAnalysis()
# bayesAnalysis()
# svcAnalysis()